### Load Data

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import ast
import openpyxl
import sys


import cred
import Constants

In [ ]:
ingestion_data = pd.read_excel('archway_legacy_projects_postTransformation.xlsx', sheet_name='Primary')

### Data Cleaning and Transformation

In [ ]:
def clean_community_org(value):
    if pd.isna(value):
        return value
    
    value = str(value).strip()
    # Check if value is in list format: ['text']
    if value.startswith('[') and value.endswith(']'):
        # Extract content between brackets
        value = value[1:-1].strip()
        # Remove quotes if present
        if value.startswith("'") and value.endswith("'"):
            value = value[1:-1]
        elif value.startswith('"') and value.endswith('"'):
            value = value[1:-1]
    return value
ingestion_data['Community Organizations'] = ingestion_data['Community Organizations'].apply(clean_community_org)

In [ ]:
def clean_list(value):
    if pd.isna(value):
        return value
    value = value.replace(';', ',')
    return ast.literal_eval(value)
ingestion_data['Semester(s)'] = ingestion_data['Semester(s)'].apply(clean_list)
ingestion_data['Client Type'] = ingestion_data['Client Type'].apply(clean_list)
ingestion_data['Priority Area'] = ingestion_data['Priority Area'].apply(clean_list)
ingestion_data['QPM Topics'] = ingestion_data['QPM Topics'].apply(clean_list)
ingestion_data['Project Unit Association'] = ingestion_data['Project Unit Association'].apply(clean_list)
ingestion_data['Associated Locations'] = ingestion_data['Associated Locations'].apply(clean_list)

In [ ]:
# Add this function to clean text for JSON
def clean_for_json(text):
    if pd.isna(text):
        return ""
    text = str(text).strip()

    # More comprehensive character replacement/removal - be aggressive
    text = text.replace("\\", "")  # Escape backslashes
    text = text.replace('"', "'")    # Escape double quotes
    text = text.replace('\n', ' ')     # Newlines to spaces
    text = text.replace('\r', ' ')     # Carriage returns to spaces
    text = text.replace('\t', ' ')     # Tabs to spaces
    text = text.replace('\u201c', "'").replace('\u201d', "'") # smart quotes to regular quotes
    text = text.replace('\u2018', "'").replace('\u2019', "'") # more smart quotes
    text = text.replace('\xa0', ' ') # Non-breaking space to regular space
    text = ''.join(c for c in text if c.isprintable()) # Remove non-printable characters
    
    # Limit length if needed
    # if len(text) > 1999:  # Choose appropriate limit
    #     text = text[:1999] + "..."
        
    return text
  
ingestion_data['Project Details'] = ingestion_data['Project Details'].apply(clean_for_json)

In [ ]:

ingestion_data.head()
print(ingestion_data.iloc[11]['Project Details'])


### Fetch Column Details

In [ ]:
# Print Cloumn Details
def get_column_details(board_id='8918391890'):
    """
    Fetch column details from a Monday.com board
    
    Args:
        board_id (str): The ID of the Monday.com board
        
    Returns:
        list: List of dictionaries containing column information (id, title, type)
    """
    # Define the URL and headers for the request
    headers = {
        "Authorization": cred.API_TOKEN,
        "Content-Type": "application/json"
    }
    
    # Define the query to fetch column names and descriptions (types)
    query = """
    {
      boards (ids: %s) {
        columns {
          id
          title
          type
        }
      }
    }
    """ % board_id
    
    # Make the POST request to the Monday.com API
    response = requests.post(Constants.URL, headers=headers, json={"query": query})
    
    # Process the response
    data = response.json()
    columns = data['data']['boards'][0]['columns']
    
    print(f"Columns for board ID: {board_id}")
    for column in columns:
      print(f"Column ID: {column['id']}, Title: {column['title']}, Type: {column['type']}")
        

get_column_details()

### Cache for All the DropDowns

In [ ]:
def fetch_dropdown_options(column_id, board_id='8918391890'):
    """
    Fetch dropdown options from a Monday.com board column
    
    Args:
        board_id (str): Monday.com board ID
        column_id (str): ID of the dropdown column to fetch
        
    Returns:
        dict: Dictionary mapping dropdown label names to their IDs
    """
    headers = {
        "Authorization": cred.API_TOKEN,
        "Content-Type": "application/json"
    }
    
    # Define the query to fetch the dropdown column details
    query = """
    {
      boards (ids: %s) {
        columns (ids: "%s") {
          id
          title
          type
          settings_str
        }
      }
    }
    """ % (board_id, column_id)
    
    # Make the POST request to the Monday.com API
    response = requests.post(Constants.URL, headers=headers, json={"query": query})
    
    # Parse the response
    data = response.json()
    column_data = data['data']['boards'][0]['columns'][0]
    
    # Extract the dropdown options from the settings_str field
    dropdown_options = json.loads(column_data['settings_str'])
    
    # Create a dictionary mapping label names to IDs
    label_to_id = {}
    for option in dropdown_options['labels']:
        label_to_id[option['name']] = option['id']
    
    return label_to_id

In [ ]:
# Cache for Project Location
project_location_cache = fetch_dropdown_options('dropdown__1')

# Cache for Semester
semester_cache = fetch_dropdown_options('dropdown_mkpyjqsg')

# Cache for Priority Area
priority_area_cache = fetch_dropdown_options('dropdown_mkpybd3z')

# Cache for QPM Topics
qpm_topics_cache = fetch_dropdown_options('dropdown14__1')

# Cache for Client type
client_type_cache = fetch_dropdown_options('dropdown01__1')

# Cache for Project Unit Association
project_unit_association_cache = fetch_dropdown_options('dropdown07__1')

# Cache for Associated Location
associated_location_cache = fetch_dropdown_options('dropdown2__1')

print(project_location_cache)
print(semester_cache)
print(priority_area_cache)
print(qpm_topics_cache)
print(client_type_cache)
print(project_unit_association_cache)
print(associated_location_cache)

### Function to create entry in Monday.com board

In [ ]:
def create_monday_item(item_name, column_values):
    """
    Create a new item in a Monday.com board
    
    Args:
        board_id (str): The ID of the Monday.com board
        item_name (str): Name of the item to create
        column_values (dict): Dictionary of column IDs and their values
        
    Returns:
        dict: The response from the Monday.com API
    """
    headers = {
        "Authorization": cred.API_TOKEN,
        "Content-Type": "application/json"
    }
    
    BOARD_ID = '8918391890'

    # Convert the dictionary to a JSON string and Escape the JSON string for GraphQL
    column_values_json = json.dumps(column_values).replace('"', '\\"')

    # Define the query to create an item with multiple column values
    query = f"""
    mutation {{
      create_item (
        board_id: {BOARD_ID},
        item_name: "{item_name}",
        column_values: "{column_values_json}"
      ) {{
        id
      }}
    }}
    """
    # % (BOARD_ID, item_name, column_values_json)

    # Make the POST request to the Monday.com API
    response = requests.post(Constants.URL, headers=headers, json={"query": query})

    # Return the response as a dictionary
    return response.json()

In [ ]:
def process_dropdown_list(row, column_name, cache_dict, item_name):
    """
    Process a list column from the DataFrame and map values to Monday.com dropdown IDs
    
    Args:
        row (pandas.Series): DataFrame row
        column_name (str): Column name containing list values
        cache_dict (dict): Mapping of dropdown option names to their IDs
        item_name (str): Item name for error reporting
        
    Returns:
        dict or None: Monday.com formatted dropdown value dictionary or None if no valid IDs
    """
    # Get the value and check its type
    value = row[column_name]
    # Check if it's a list-like structure with contents
    if isinstance(value, list) and len(value) > 0:
        ids = []
        # Loop through each value in the list
        for element in row[column_name]:
            # Look up ID in cache dictionary
            value_id = cache_dict.get(element)
            if value_id:
                ids.append(value_id)
            else:
                print(f"Error: {column_name} value '{element}' not found in dropdown cache for Project: {item_name}")
                sys.exit(1)
        
        # Only return values if we found any valid IDs
        if ids:
            return {"ids": ids}
    
    return None

In [ ]:
# counter=0
# Loop through each row in your DataFrame
for index, row in ingestion_data.iloc[0:10].iterrows():
    # Extract item name from a specific column (adjust column name as needed)
    item_name = row['Project Name']  # Replace with your actual column name for project name
    
    # Initialize empty column values dictionary
    column_values = {}
    
    # Project Code (always include as it's likely a required identifier)
    if pd.notnull(row['Project Code']) and str(row['Project Code']).strip():
        column_values['text3__1'] = str(row['Project Code'])
    else:
        print(f"Error: Project Code is missing or empty for Project: {item_name}")
        sys.exit(1)
    
    # Status (always include as it's required)
    if pd.notnull(row['Status']) and str(row['Status']).strip():
        column_values["status"] = {"label": str(row['Status']).strip()}
    else:
        print(f"Error: Status is missing or empty for Project: {item_name}")
        sys.exit(1)
    
    # Project Link
    if pd.notnull(row['Project Link']) and str(row['Project Link']).strip():
        column_values["link__1"] = {"url": str(row['Project Link']).strip(),"text": item_name}
    
    # Project Description - only add if not empty
    if pd.notnull(row['Project Details']) and str(row['Project Details']).strip():
        column_values["long_text__1"] = str(row['Project Details']).strip()
    
    # Community Organization - only add if not empty
    if pd.notnull(row['Community Organizations']) and str(row['Community Organizations']).strip():
        column_values["text2__1"] = str(row['Community Organizations']).strip()
    
    # Number fields - only add if they have values
    if pd.notnull(row['Number of Students']) and int(row['Number of Students']) > 0:
        column_values["numbers__1"] = int(row['Number of Students'])
    
    if pd.notnull(row['Number of Core Team Participants Receiving Non-Training Service (executive committee members)']) and int(row['Number of Core Team Participants Receiving Non-Training Service (executive committee members)']) > 0:
        column_values["numbers4__1"] = int(row['Number of Core Team Participants Receiving Non-Training Service (executive committee members)'])
    
    if pd.notnull(row['Number of Project Participants Receiving Non-Training Service (issue work group/community members)']) and int(row['Number of Project Participants Receiving Non-Training Service (issue work group/community members)']) > 0:
        column_values["numbers2__1"] = int(row['Number of Project Participants Receiving Non-Training Service (issue work group/community members)'])
    
    if pd.notnull(row['Project Value']) and float(row['Project Value']) > 0:
        column_values["numbers1__1"] = float(row['Project Value'])
    
    if pd.notnull(row['Contract Amount']) and float(row['Contract Amount']) > 0:
        column_values["numbers6__1"] = float(row['Contract Amount'])
    
    if pd.notnull(row['Grant Amount']) and float(row['Grant Amount']) > 0:
        column_values["numbers8__1"] = float(row['Grant Amount'])
        
    ## Dropdown fields - only add if they have values
    if pd.notnull(row['Location']) and str(row['Location']).strip():
        location_id = project_location_cache.get(str(row['Location']).strip())
        if location_id:
            column_values["dropdown__1"] = {"ids": [location_id]}
        else:
            print(f"Error: Location '{str(row['Location']).strip()}' not found in project location cache for Project Name: {item_name}")
            sys.exit(1)
    
    # Dropdown field Semester(s)
    semester_values = process_dropdown_list(
        row=row, 
        column_name='Semester(s)', 
        cache_dict=semester_cache, 
        item_name=item_name)
    if semester_values:
        column_values["dropdown_mkpyjqsg"] = semester_values
        
    # Dropdown field Client Type
    client_type_values = process_dropdown_list(
        row=row, 
        column_name='Client Type', 
        cache_dict=client_type_cache, 
        item_name=item_name)
    if client_type_values:
        column_values["dropdown01__1"] = client_type_values
        
    # Dropdown field Priority Area
    priority_area_values = process_dropdown_list(
        row=row, 
        column_name='Priority Area', 
        cache_dict=priority_area_cache, 
        item_name=item_name)
    if priority_area_values:
        column_values["dropdown_mkpybd3z"] = priority_area_values
        
    # Dropdown field QPM Topics
    qpm_topics_values = process_dropdown_list(
        row=row, 
        column_name='QPM Topics', 
        cache_dict=qpm_topics_cache, 
        item_name=item_name)    
    if qpm_topics_values:
        column_values["dropdown14__1"] = qpm_topics_values
        
    # Dropdown field Project Unit Association
    project_unit_association_values = process_dropdown_list(
        row=row, 
        column_name='Project Unit Association', 
        cache_dict=project_unit_association_cache, 
        item_name=item_name)
    if project_unit_association_values:
        column_values["dropdown07__1"] = project_unit_association_values
        
    # Dropdown field Associated Locations
    associated_locations_values = process_dropdown_list(
        row=row, 
        column_name='Associated Locations', 
        cache_dict=associated_location_cache, 
        item_name=item_name)
    if associated_locations_values:
        column_values["dropdown2__1"] = associated_locations_values
    
    # Call the function to create item in Monday.com
    try:
        response = create_monday_item(item_name, column_values)
    except Exception as e:
        print(f"Exception creating item: {item_name}")
        print(f"Error details: {e}")
        print("Stopping processing due to API error")
        break
    
    # Check if the API call was successful
    if 'errors' in response or response.get('data') is None:
        print(f"ERROR creating item: {item_name}")
        print(f"Error details: {response}")
        print("Stopping processing due to API error")
        break
    
    # Print the response or handle errors
    print(f"Created item {index+1}: {item_name}, Response: {response}")
    
    # counter += 1
    # if counter >= 2:
    #     break